# Traitement des données de financements 

In [1]:
#importer les packages
import requests
import pandas as pd
import numpy as np
from code_utils.functions import aplatir,address,champs_dict,identifie_structure,identifie_personne,identifiant_prefere,replace_all,nettoie_scanR,orcid_to_idref,persons
from tqdm import tqdm
import pprint as pp
tqdm.pandas()
from code_utils.pickle import load_cache,write_cache
import os
from dotenv import load_dotenv
import json

load_dotenv()

Authorization = os.getenv('Authorization')
Authorization_ORCID = os.getenv('Authorization_ORCID')

In [2]:
sources={'ANR':{'url_partenaires':"https://www.data.gouv.fr/fr/datasets/r/9b08ee21-7372-47a4-9831-4c56a8099ee8",
                'url_projets': "https://www.data.gouv.fr/fr/datasets/r/afe3d11b-9ea2-48b0-9789-2816d5785466",
                'cache_structure':'cached_anr_data',
                'nom':'Projet.Partenaire.Responsable_scientifique.Nom',
                'prenom':'Projet.Partenaire.Responsable_scientifique.Prenom',
                'nom_structure':'Projet.Partenaire.Nom_organisme',
                'adresse':'',
                'ville':'Projet.Partenaire.Adresse.Ville',
                'region':'',
                'pays':'Projet.Partenaire.Adresse.Pays',
                'code_projet':'Projet.Code_Decision_ANR',
                'annee':'AAP.Edition',
                'identifiants_preferes_structure':[],
                'identifiants_preferes_personne':['id_personne','idref_ORCID'],
                'id_ORCID':'Projet.Partenaire.Responsable_scientifique.ORCID',
                'titre_fr':'',
                'titre_en':'',
                'resume_fr':'',
                'resume_en':'',
                'budget':'',
                'acronyme':''
                },
        'ANSES':{'url_partenaires':"https://www.data.gouv.fr/fr/datasets/r/d0b0b666-d997-4ffa-b501-7a3d81dcdf32",
                'url_projets': "https://www.data.gouv.fr/fr/datasets/r/d6d580fb-9703-4f39-8df2-585e44594091",
                'cache_structure':'cached_anses_data',
                'nom':'Nom Responsable Scientifique',
                'prenom':'Prénom Responsable Scientifique',
                'nom_structure':'Organisme',
                'adresse':'',
                'ville':'Ville',
                'region':'',
                'pays':'Pays',
                'code_projet':'code convention homogénéisé',
                'annee':'Code convention',
                'identifiants_preferes_structure':[],
                'identifiants_preferes_personne':['id_personne'],
                'id_ORCID':'',
                'titre_fr':'',
                'titre_en':'',
                'resume_fr':'',
                'resume_en':'',
                'budget':'',
                'acronyme':''
                },
        'IRESP':{'url_partenaires2':"https://www.data.gouv.fr/fr/datasets/r/67744c92-39b9-450c-8f6c-296d47fa90a4",
                'url_partenaires1':"https://www.data.gouv.fr/fr/datasets/r/ec2623d0-b0e8-4269-a1c8-3d8c361e71ee",
                'url_projets1': "https://www.data.gouv.fr/fr/datasets/r/82b91478-a09d-4f2c-be1b-1a034b7fa909",
                'url_projets2': "https://www.data.gouv.fr/fr/datasets/r/bfc1540e-8f6e-427f-bd81-7f89b72f5599",
                'cache_structure':'cached_iresp_data',
                'nom':'Nom',
                'prenom':'Prenom',
                'nom_structure':'Nom_equipe',
                'adresse':'',
                'ville':'Ville',
                'region':'',
                'code_projet':'ID_Projet',
                'identifiants_preferes_structure':[],
                'identifiants_preferes_personne':['id_personne','idref_ORCID'],
                'id_ORCID':'ORCID',
                'titre_fr':'',
                'titre_en':'',
                'resume_fr':'',
                'resume_en':'',
                'budget':'',
                'acronyme':''
                },
        'SIRANO':{'url_partenaires':"https://www.data.gouv.fr/fr/datasets/r/472569d2-0704-468e-bda0-7665a1f3c33c",
                'url_projets': "https://www.data.gouv.fr/fr/datasets/r/472569d2-0704-468e-bda0-7665a1f3c33c",
                'cache_structure':'cached_sirano_data',
                'nom':'nom_porteur',
                'prenom':'prenom_porteur',
                'nom_structure':'nom_etablissement',
                'adresse':'',
                'region':'region',
                'code_projet':'appel_a_projets',
                'annee':'annee_de_selection',
                'identifiants_preferes_structure':[],
                'identifiants_preferes_personne':['id_personne'],
                'id_ORCID':'',
                'titre_fr':'',
                'titre_en':'',
                'resume_fr':'',
                'resume_en':'',
                'budget':'financement_total',
                'acronyme':''
                },
        'REG_IDF':{'url_projets':"https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/dim_map_projets_finances/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B",
                'url_partenaires': "https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/dim_map_projets_finances_entites_partenariat/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B",
                'cache_structure':'cached_idf_data',
                'nom_structure':'entite_nom',
                'ville':'entite_commune',
                'pays':'entite_pays',
                'region':'',
                'adresse':'entite_adresse',
                'code_projet':'ï»¿projet_ID',
                'annee':'projet_annee_depot',
                'identifiants_preferes_structure':['entite_RNSR','entite_ROR','entite_SIRET','id_structure_matcher','id_structure_scanr','code'],
                'identifiants_preferes_personne':[],
                'id_ORCID':'',
                'titre_fr':'projet_titre-fr',
                'titre_en':'projet_titre-en',
                'resume_fr':'projet_resume-fr',
                'resume_en':'projet_resume-en',
                'budget':'projet_subvention_allouee',
                'acronyme':'projet_acronyme'
                }      
                }

In [ ]:
source=list(sources.keys())[4]
source

In [ ]:
#cache structures, personnes et orcid avec differentes sources de donnees
cached_data = {}
try:
    cached_data = load_cache(cached_data,f"./{source}/caches/cached_{source.lower()}_data.pkl")
except:
    write_cache(cached_data,f"./{source}/caches/cached_{source.lower()}_data.pkl")
    
cached_data_persons = {}
try:
    cached_data_persons = load_cache(cached_data_persons,f"./{source}/caches/cached_{source.lower()}_data_persons.pkl")
except:
    write_cache(cached_data_persons,f"./{source}/caches/cached_{source.lower()}_data_persons.pkl")
    
cached_data_orcid = {}
try:
    cached_data_orcid = load_cache(cached_data_orcid,f"./{source}/caches/cached_{source.lower()}_data_orcid.pkl")
except:
    write_cache(cached_data_orcid,f"./{source}/caches/cached_{source.lower()}_data_orcid.pkl")

# Données partenaires

In [7]:
# amener les partenaires depuis le site 
if source=='ANR':
    page_partenaires_10 = requests.get(sources[source]['url_partenaires']).json()
    colonnes_partenaires_10 = page_partenaires_10['columns']
    donnees_partenaires_10 = page_partenaires_10['data']
    df_partenaires=pd.DataFrame(data=donnees_partenaires_10,columns=colonnes_partenaires_10)
elif source=='ANSES':
    df_from_anses=pd.read_excel(sources[source]['url_partenaires'])
    df=df_from_anses.iloc[1:,:]
    df.columns=list(df_from_anses.iloc[0,:])
    dict_equipe={list(df_from_anses.columns)[k].replace('Équipe 10 ','Équipe 10').replace('Équipe13','Équipe 13'):k for k in range (len(list(df_from_anses.columns))) if list(df_from_anses.columns)[k].find('Équipe')>=0}
    list_df=[]
    number=3
    for n in range(1,len(dict_equipe)+1):
        equipe_n=pd.concat([df.iloc[:,0:3],df.iloc[:,number:number+6]], axis=1)
        list_df.append(equipe_n)
        number+=6
    df_partenaires=pd.concat([list_df[k].dropna(subset=[sources[source]['nom'], sources[source]['prenom'],sources[source]['nom_structure'], sources[source]['nom'], 'Pays'], how='all') for k in range(len(list_df))])
    
elif source=='IRESP':
    df_partenaires1=pd.read_csv(sources[source]['url_partenaires1'] ,sep=";", encoding='ISO-8859-1')
    df_partenaires2=pd.read_csv(sources[source]['url_partenaires2'] ,sep=";", encoding='ISO-8859-1')
    df_partenaires=pd.concat([df_partenaires1,df_partenaires2])
else:
    df_partenaires=pd.read_csv(sources[source]['url_partenaires'] ,sep=";", encoding='ISO-8859-1')

df_partenaires=df_partenaires.reset_index()
del df_partenaires['index']

# Matcher établissement

In [8]:
id_struct=df_partenaires
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
id_struct=id_struct.drop_duplicates(subset=[f"{sources[source]['nom_structure']}2"])

In [ ]:
id_struct.progress_apply(lambda row: identifie_structure(row,source,cached_data,sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays'],sources[source]['code_projet'],False), axis=1) 
write_cache(cached_data,f"./{source}/caches/cached_{source}_data.pkl")

In [ ]:
id_struct['id_structure_matcher']=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: cached_data[x])

In [11]:
id_struct=id_struct.reset_index()
del id_struct['index']
id_struct.to_excel(f"./{source}/df_partenaires.xlsx")
id_struct.to_json(f"./{source}/df_partenaires.json")

In [ ]:
id_struct=pd.read_json(f"./{source}/df_partenaires.json")
id_struct=id_struct[[sources[source]['nom_structure'],'id_structure_matcher']]
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))

df_partenaires[f"{sources[source]['nom_structure']}2"]=df_partenaires.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
df_partenaires_struct=pd.merge(df_partenaires,id_struct[[f"{sources[source]['nom_structure']}2",'id_structure_matcher']], on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#compléter les données avec scanR
url_scanr='https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/scanR/projects.json'
requete_scanR = requests.get(url_scanr)
page_scanR= requete_scanR.json()
df_scanR=pd.DataFrame(page_scanR)
scanR=df_scanR.explode('participants').loc[:,['id','participants']]
scanR=scanR.rename(columns={'id':'id_anr'})
scanR['index']=[x for x in range(len(scanR))]
scanR=scanR.set_index('index')
scanR['id_structure_scanr']=scanR['participants'].apply(lambda x: x.get(str('structure')) if isinstance(x, dict) else None )
scanR['nom_struct']=scanR['participants'].apply(lambda x: nettoie_scanR(x))
del scanR['participants']
scanR_nettoye=scanR.drop_duplicates(subset='nom_struct')
scanR_nettoye[f"{sources[source]['nom_structure']}2"]=scanR_nettoye.loc[:,'nom_struct'].apply(lambda x: replace_all(str(x).lower()))
scanR_nettoye=scanR_nettoye[['id_structure_scanr',f"{sources[source]['nom_structure']}2"]]
scanR_nettoye=scanR_nettoye.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")

In [ ]:
df_partenaires_struct=pd.merge(df_partenaires_struct,scanR_nettoye, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#######fichier avec les identifiants structures rettrouvés à la main par Emmanuel ==> 'code'
scanr_structures=pd.read_excel('scanr_partenaires_non_identifies.xlsx')
scanr_structures[f"{sources[source]['nom_structure']}2"]=scanr_structures.loc[:,'Nom'].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
scanr_structures=scanr_structures[[f"{sources[source]['nom_structure']}2",'code']]
scanr_structures=scanr_structures.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
df_partenaires_complet=pd.merge(df_partenaires_struct,scanr_structures, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_complet

In [14]:
if 'finess' in list(df_partenaires.columns):
    finess_siret=pd.read_json("finess_siret-siege.json")
    df_partenaires_complet=pd.merge(df_partenaires_complet,finess_siret,how='left', on='finess')

In [ ]:
sources[source]['identifiants_preferes_structure']

In [ ]:
df_partenaires_complet['id_structure']=df_partenaires_complet.apply(lambda row: identifiant_prefere(row,sources[source]['identifiants_preferes_structure']), axis=1)
df_partenaires_complet

In [17]:
df_partenaires_complet.to_excel(f"./{source}/df_partenaires_id_structures.xlsx")
df_partenaires_complet.to_json(f"./{source}/df_partenaires_id_structures.json")

In [18]:
########récupération des structures sans identifiants pour les donner à Emmanuel
identifiants_a_remplir=df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]
identifiants_a_remplir
identifiants_a_remplir=identifiants_a_remplir.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
identifiants_a_remplir=identifiants_a_remplir.reset_index()

In [ ]:
identifiants_a_remplir

In [ ]:
if sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['adresse'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['region'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['region']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville']]]
identifiants_a_remplir

In [21]:
identifiants_a_remplir.to_excel(f"./structures_manquantes/partenaires_non_identifies_{source}.xlsx", index=False)

# Matcher des chercheurs

In [22]:
df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_structures.json")

In [23]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: identifie_personne(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)
    write_cache(cached_data_persons,f"./{source}/caches/cached_data_persons.pkl")
    df_partenaires.to_excel(f"./{source}/df_partenaires_id_personne.xlsx")
    df_partenaires.to_json(f"./{source}/df_partenaires_id_personne.json")

In [24]:
if sources[source]['id_ORCID'] in list(df_partenaires.columns):
    df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_personne.json")
    df_partenaires['idref_ORCID']=df_partenaires.progress_apply(lambda row: orcid_to_idref(row,cached_data_orcid,sources[source]['id_ORCID'],Authorization_ORCID), axis=1)
    write_cache(cached_data_orcid,f"./{source}/caches/cached_data_orcid.pkl")
    df_partenaires.to_excel(f"./{source}/df_partenaires_id_personne_ORCID.xlsx")
    df_partenaires.to_json(f"./{source}/df_partenaires_id_personne_ORCID.json")

# ENVOI DES PROJETS SUR SCANR

In [5]:
if len(sources[source]['identifiants_preferes_personne'])==2:
    df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_personne_ORCID.json")
elif len(sources[source]['identifiants_preferes_personne'])==1:
    df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_personne.json")
else:
    df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_structures.json")

In [6]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_person']=df_partenaires.apply(lambda row: identifiant_prefere(row,sources[source]['identifiants_preferes_personnes']), axis=1)
    df_partenaires['persons']=df_partenaires.progress_apply(lambda row: persons(row) ,axis=1)
else:
    df_partenaires['persons']=np.nan

In [ ]:
df_partenaires

In [8]:
if source != 'SIRANO':
    df_partenaires=df_partenaires.groupby([sources[source]['code_projet']]).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False).reset_index()
else:
    df_projets=df_partenaires.groupby([sources[source]['code_projet'], sources[source]['annee'], sources[source]['acronyme'],sources[source]['titre'],sources[source]['budget']], dropna=False).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False)

In [21]:
# amener les projets depuis le site 
if source=='ANR':
    page_projets_10 = requests.get(sources[source]['url_projets']).json()
    colonnes_projets_10 = page_projets_10['columns']
    donnees_projets_10 = page_projets_10['data']
    df_projets=pd.DataFrame(data=donnees_projets_10,columns=colonnes_projets_10)
elif source=='IRESP':
    df_projets1=pd.read_csv(sources[source]['url_projets1'] ,sep=";", encoding='ISO-8859-1')
    df_projets2=pd.read_csv(sources[source]['url_projets2'] ,sep=";", encoding='ISO-8859-1')
    df_projets=pd.concat([df_projets1,df_projets2])
elif source!='SIRANO':
    df_projets=pd.read_csv(sources[source]['url_projets'] ,sep=";", encoding='ISO-8859-1')

df_projets=df_projets.reset_index()
del df_projets['index']

In [ ]:
if source!='SIRANO':
    df_projets=pd.merge(df_projets,df_partenaires,on=sources[source]['code_projet'], how='left')
else :
    df_projets['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[sources[source]['annee']]}-{row[sources[source]['acronyme']]}" , axis=1)
    del df_projets['code_projet']
    sources[source]['code_projet']='id'

df_projets['type']=source
df_projets['name']=df_projets.progress_apply(lambda row: champs_dict(row,sources[source]['titre_fr'],sources[source]['titre_en']) ,axis=1)
df_projets['description']=df_projets.progress_apply(lambda row: champs_dict(row,sources[source]['resume_fr'],sources[source]['resume_en']) ,axis=1)
df_projets.loc[:,sources[source]['budget']]=df_projets.loc[:,sources[source]['budget']].apply(lambda x : float(str(x).replace('.0','').replace('.00','').replace(' ','').replace(',','.').replace('€','')))
df_projets=df_projets.rename(columns={sources[source]['annee']:'year',sources[source]['acronyme']:'acronym',
                                      sources[source]['budget']:'budget_financed',sources[source]['code_projet']:'id'})
df_projets=df_projets[['id','type','name','description','acronym','year','budget_financed','persons']]

In [ ]:
df_projets[df_projets.duplicated(subset=['id'])]

In [ ]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

In [15]:
#envoi
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id') for x in err]).drop_duplicates().tolist() 

# ENVOI DES PARTENAIRES SUR SCANR

In [46]:
df_partenaires=pd.read_json(f"./{source}/df_partenaires_id_structures.json")

In [47]:
### ATTENTION, vérifier que les projets sirano sont dans des structures françaises
if source=='IRESP':
    df_partenaires[sources[source]['pays']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[1].replace(')','') if x.find('(')>=0 else 'France')
    df_partenaires.loc[:,sources[source]['ville']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[0] if x.find('(')>=0 else x)
  
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires.loc[:,'id_structure']=df_partenaires.loc[:,'id_structure'].apply(lambda x: x[0] if isinstance(x,list) else x )

In [ ]:
if source in ['ANSES','SIRANO']:
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[{sources[source]['nom_structure']}+'2']}-{row[sources[source]['nom']]}-{row[sources[source]['prenom']]}" , axis=1)
if source =='REG_IDF':
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row['entite_role']}" , axis=1)
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires=df_partenaires.rename(columns={sources[source]['nom_structure']: 'name', sources[source]['code_projet']: 'project_id', 'id_structure':'participant_id'})
df_partenaires=df_partenaires[['name','id','project_id','participant_id','address']]
df_partenaires['project_type']=source
df_partenaires['participant_id']=df_partenaires.loc[:,'participant_id'].apply(lambda x: str(x[0]).replace('.0','') if isinstance(x,list) else str(x).split(';')[0].replace('.0',''))
df_partenaires=df_partenaires[['id','project_id', 'project_type', 'participant_id', 'name','address']]
df_partenaires['name'] = df_partenaires['name'].astype(str)
df_partenaires

In [ ]:
df_partenaires[df_partenaires.duplicated(subset=['id'])]

In [ ]:
dict_row=df_partenaires.iloc[1,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
dict_row2

In [ ]:
len(df_partenaires)

In [54]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id')[25:] for x in err]).drop_duplicates().tolist()